<a href="https://colab.research.google.com/github/wanderson42/Portfolio-DS/blob/main/Projeto_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Data Science - Turing USP** 

# **Projeto de Portfólio Nº 2:** 

<span style='color:Gray'>  Data Visualization com Plotly em um Netflix Dataset </span>

Autor: Wanderson Ferreira

## **1. Introdução: Algumas informações sobre a Netflix e Descrição do Dataset**


</div>
<center><img src="https://assets.datacamp.com/production/project_1237/img/netflix.jpg" alt="Someone's feet on table facing a television"width="900"/> <center> </div>


Netflix é uma provedora global de filmes e séries de televisão via streaming sediada em Los Gatos, Califórnia, e que atualmente possui mais de 220 milhões de assinantes. Dado o grande número de filmes e séries disponíveis na plataforma, é uma oportunidade perfeita para colocar em prática diversas técnicas de Vizualização de dados!

**Descrição do dataset:**

Este dataset contém dados coletados da Netflix de diferentes Filmes e Séries de TV entre o ano de 2008 a 2021.

> - **show_id:** ID exclusivo para cada Filme ou Série de TV;
>
> - **type:** Identificador - que mostra se é conteúdo é um filme ou programa de TV;
>
> - **title:** Título do Filme / Série de TV;
>
> - **director:** Diretor do Filme / Série de TV;
>
> - **cast:** Atores envolvidos no Filme / Série de TV;
>
> - **country:** País onde o Filme / Série de TV foi produzido;
>
> - **date_added:** Data em que o Filme / Série de TV foi lançado na Netflix;
>
> - **release_year:** Ano real em que o Filme / Série de TV conteúdo foi lançado;
>
> - **rating:** Classificação indicativa do Filme / Série de TV;
>
> - **duration:** Duração total - em minutos do Filme ou número de temporadas da Série de TV;
>
> - **listed_in:** Gênero;
>
> - **description:** Sinopse.


## **2. Importando Bibliotecas**

In [12]:
import pandas as pd
import numpy as np
import plotly.express as px
colors = ['#0F2C67', '#CD1818', '#F3950D', '#0B4619']

## **3. Carregando o dataset e obtendo primeiras impressões**


In [13]:
#Dataset
url = 'https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/netflix_titles.csv'
df = pd.read_csv(url)

#Primeiras impressões sobre o dataframe
df.info()

# Obtendo-se uma amostra do datset 
df.sample(n = 4).style.background_gradient(cmap = 'Set3')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8579,s8580,Movie,Thong Dee Fun Khao,Bin Bunluerit,"Buakaw Banchamek, Sornsin Maneewan, Phutharit Prombandal, Nantawut Boonrubsub, Vannapoom Songsuparp, Chutirada Junthit, Jaran Ngamdee, Kochakorn Nimakorn, Rapeepat Ekpankul, Manop Aswathep",Thailand,"January 15, 2018",2017,TV-MA,122 min,"Action & Adventure, International Movies, Sports Movies","After losing a boxing match, a talented Muay Thai fighter leaves home to train with the best. On his quest, he finds friends, foes and lots of action."
8047,s8048,Movie,Socha Na Tha,Imtiaz Ali,"Abhay Deol, Ayesha Takia, Apoorva Jha, Ayesha Jhulka, Rati Agnihotri, Sandhya Mridul, Suresh Oberoi, Manish Chaudhary",India,"July 1, 2018",2005,TV-14,137 min,"Comedies, International Movies, Romantic Movies","An arranged marriage becomes a friendship for Viren and Aditi, who aren't in love – until they begin having unexpected feelings for each other."
178,s179,Movie,The Interview,"Evan Goldberg, Seth Rogen","James Franco, Seth Rogen, Lizzy Caplan, Randall Park, Diana Bang, Timothy Simons, Reese Alexander, James Yi",United States,"September 1, 2021",2014,R,112 min,"Action & Adventure, Comedies",Seth Rogen and James Franco star in this provocative comedy about two journalists recruited by the CIA after they arrange an interview with Kim Jong-un.
6892,s6893,Movie,Grass,Tanuj Chopra,"Emily Chang, Pia Shah, Anthony Ma",United States,"February 14, 2019",2017,TV-MA,57 min,"Comedies, Independent Movies",Good girl Cam recruits her pot-loving bestie Jinky to help deliver a backpack full of weed for her drug-dealing fiancé.


## **4. Data Preprocessing - Feature Engineering: Uma inspeção dos nossos dados**

Aqui implementei algumas ferramentas de preprocessamento da dados para flexibilizar ao máximo a abstração de insights através da vizualização dos dados. 

**4.1 Data Cleaning**

Nessa etapa foi feita uma varredura em cada coluna do dataset para detectar valores ausentes, pois estes podem afetar grosseiramente a abstração de insights. 

In [14]:
def f1():  
  fig = px.imshow(df.isnull(), title='Mapa de Missing Values'); 
  fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20))
  return fig.show()

def f2():
  # Calculando o valor total de Nan values no dataset de treino
  df_total_nan = df.isnull().sum()
  df_percent_nan = ((df.isnull().sum()/df.shape[0])*100)

  # Construindo uma tabela de missing values 
  table_missing_data = pd.concat([df_total_nan,df_percent_nan],
                                axis=1, 
                                keys=['NaN', 'NaN(%)'],
                                sort = True)

  # Somando todos os missing values de ambas as colunas
  a = table_missing_data['NaN'].sum()
  b = a*100/(df.shape[0])
  # Adicionando uma linha contento os valores totais
  row = pd.Series({'NaN':a,'NaN(%)':b},name='Total')
  table_missing_data = table_missing_data.append(row)
  table_missing_data = table_missing_data.astype(int)

  return table_missing_data.style.bar(color = 'lightcoral')

f1(); f2()

,NaN,NaN(%)
cast,825,9
country,831,9
date_added,10,0
description,0,0
director,2634,29
duration,3,0
listed_in,0,0
rating,4,0
release_year,0,0
show_id,0,0


> - A partir dos resultados acima, observa-se uma quantidade expressiva de valores ausentes, onde **48\%** das linhas do dataset contêm algum dado faltante ao longo das features. Com destaque para feature `director`, onde concentra-se a maior quantidade desses valores ausentes. 

A imputação é o método de preprocessamento mais comum para preencher valores ausentes. Aqui, utilizarei a função `fillna` do Pandas para a tarefa de imputação. E também utilizarei a função `dropna` para deletar certas linhas que contem valores ausentes. 

In [15]:
df.director.fillna("No Director", inplace=True)
df.cast.fillna("No Cast", inplace=True)
df.country.fillna("Country Unavailable", inplace=True)
df.dropna(subset=["date_added", "duration", "rating"], inplace=True)

Uma vez que `director`, `cast` e `country` contêm a maioria dos valores nulos, por simplicidade poderiamos remove-los, contudo, isto não seria benéfico para a nossa EDA, uma vez que se trataria de uma grande perda de informação. Assim optamos por tratar cada valor em falta como "Não disponível". Já nas features `date_added`, `duration` e `rating`, dada a quantidade insignificante de valores ausentes, pode-se remove-los sem grandes problemas. Finalmente, ao executar a célula acima podemos ver que não há mais valores ausentes: 

In [16]:
df.isnull().any()

show_id         False
type            False
title           False
director        False
cast            False
country         False
date_added      False
release_year    False
rating          False
duration        False
listed_in       False
description     False
dtype: bool

**4.2 Data Transformation**

Nessa etapa novas features foram extraidas a partir de `date_added`:

In [17]:
# Convertendo "date_added" para o formato datetime
df["date_added"] = pd.to_datetime(df['date_added'])
# Aplicando atributos dt.year e dt.mounth para extrair ano e mês e incorporar tudo em duas novas colunas 
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

#Substituindo o número do mês pelo nome do mês para uma melhor visualização:
df['month_added'] = df['month_added'].replace({1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'June', 7:'July', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'})


E para facilitar futuras análises a feature `rating` foi padronizada num formato mais familiar pra gente: 

In [18]:
df['rating'] = df['rating'].map({ 'TV-PG': '7','TV-MA': '18','TV-Y7-FV': '7',
                                 'TV-Y7': '7','TV-14': '16','R': '18','TV-Y': 'L','NR': '18',
                                 'PG-13': '13','TV-G': 'L','PG': '7','G': 'L','UR': '18','NC-17': '18'})

Também vamos padronizar a coluna que corresponde ao gênero cinematográfico dos titulos da Netflix para facilitar as visualizações:

In [19]:
# Renomear a coluna
df.rename(columns={'listed_in':'genre'}, inplace=True)

# Reduzindo o nome de cada gênero
df['genre'] = df['genre'].str.split(',').str[0]

Uma vez que o dataset esta suficientemente tratado. Agora vamos começar a diversão e obter alguns insights a partir dos dados. 

## **5. Data Visualization com Plotly**

Finalmente chegou a etapa divertida!

Aqui sera utilizada a biblioteca Plotly para gerar vizualizações interativas que possam nos fornecer informações valiosas sobre os dados. Agora que estamos familiarizados com o dataset, algumas questões pertinentes podem surgir, conforme a seguir:

**5.1 Qual é o tipo de conteúdo preferido da Netflix: Filmes ou Series de TV?**

Para responder essa pergunta irei contruir no Plotly um gráfico de setores, comumente chamado de gráfico de pizza:

In [20]:
fig = px.pie(df['type'].value_counts().reset_index(), values='type', names='index', 
             color_discrete_sequence=px.colors.qualitative.G10, template='plotly_dark',title='Percentual de Series de TV e Filmes')
fig.update_layout(uniformtext_minsize=18, title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 18)))

fig.show()

Esse é o mais simples e provalmente o gráfico mais importante que a gente poderia fazer a partir do dataset, pois ele nos informa sobre o que o publico prefere ver. Conforme podemos notar, 69.7% das pessoas preferem Filmes e somente 30.3% séries de TV (Isso corrresponde aproximadamente a uma proporção 2:1).
Assim, com base nessa informação a Netflix pode decidir o formato do conteúdo que eles devem priorizar para fazer a audiência feliz. 




**5.2 Qual é a preferência de conteúdo entre os assinantes, com base na Classificação Indicativa?**




In [21]:
fig = px.histogram(df, x = 'rating', color = 'type', marginal = 'box', template = 'plotly_dark', title='Distribuição de Filmes e Series de TV com base na Classificação Indicativa')
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16))
fig.show()

A partir desse gráfico de barras podemos inferir qual tipo de conteúdo é mais aclamado pelo público. E essa valiosa informação ajuda a Netflix compreender qual classificação indicativa corresponde às maiores e menores audiências. No caso, pode se destacar que:

> - Os filmes com classificação 18 têm a maior proporção entre as demais classificações indicativas, com preferência maior do publico por parte dos filmes. 
>
> - Na classificação indicativa "Livre para todos os publicos" (L) praticamente não existe preferência por parte do público entre Filmes e Séries de TV. 



**5.3 Quantos Filmes e Series foram lançados no catálogo ao longo dos anos ?**

Queremos ver qual é a distribuição de lançamentos de Filmes e Séries no catálogo da Netflix ao longo dos anos.

In [22]:
movie_count = pd.DataFrame({'count': df.where(df.release_year >= 1970).groupby('type').get_group('Movie').groupby('release_year')['title'].count().sort_values(ascending=False)})
fig = px.bar(data_frame=movie_count, x=movie_count.index, y='count', labels=movie_count.index, color='count', color_discrete_sequence=colors[1],
       template="plotly_dark", title='Distribuição de Filmes disponibilizados na Netflix com base no Ano de Lançamento')
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16))

Como podemos observar no gráfico acima, o catálogo de Filmes da Netflix aumenta exponencialmente até 2017, e no ano 2018 foi  registrado o maior aumento histórico da companhia. No entanto,  devido a pandemia de COVID-19, a produção dos filmes foi largamente reduzida.

In [23]:
movie_count = pd.DataFrame({'count': df.where(df.release_year >= 1980).groupby('type').get_group('TV Show').groupby('release_year')['title'].count().sort_values(ascending=False)})
fig = px.bar(data_frame=movie_count, x=movie_count.index, y='count', labels=movie_count.index, color='count', color_discrete_sequence=colors[1],
             template="plotly_dark", title='Distribuição de Séries de TV disponibilizados na Netflix com base no Ano de Lançamento')
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16))

Já em relação ao catalogo das Series de TV, conforme o gráfico acima, a taxa de crescimento foi menos afetada que a do catálogo dos Filmes. O que faz sentido, se comparadas aos Filmes, em geral, a produção das Séries de TV tendem a ter um investimento menor e demandam de uma infraestrutura menor.

**5.4 Quais são os melhores meses para lançar um titulo no catálogo da Netflix ?**

Essa informação é valiosa para promover o lançamento de um titulo no catálogo. E consequentemente isso vai impactar as receitas da companhia. Plotando um gráfico de Funil:

In [24]:
df_month = pd.DataFrame(df.month_added.value_counts()).reset_index().rename(columns={'index':'month','month_added':'count'})

fig = px.funnel(df_month, x='count', y='month', title='Melhores meses para lançar um conteúdo',
                      height=480, width=750, color_discrete_sequence=['#b20710'])
fig.update_traces(hovertemplate=None, marker=dict(line=dict(width=0)))
fig.update_layout(margin=dict(t=60, b=20, l=70, r=40),
                        xaxis_title=' ', yaxis_title=" ",
                        plot_bgcolor='#000000', paper_bgcolor='#000000',
                        title_font=dict(size=25, color='#8a8d93'),
                        font=dict(color='#8a8d93'),
                        hoverlabel=dict(bgcolor="black", font_size=12))
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16),showgrid=False, ticksuffix=' ', showline=True)

Como já poderiamos imaginar os meses de férias (Julho e dezembro) são os melhores para lançar um titulo, pois as pessoas tendem a passar mais tempo uns com os outros e consumindo mais conteúdo digital. 

**5.5 Qual é a duração dos titulos da Netflix ?**


In [25]:
netflix_shows=df[df['type']=='TV Show']
fig = px.violin(netflix_shows, x = 'duration', template="plotly_dark", title='Duração das Temporadas das Séries na Netflix',color_discrete_sequence=['red'])
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16))

A partir do gráfico de violino (figura acima), observa-se que a maioria das Séries não passam de uma ou duas temporadas. 



In [26]:
netflix_movies=df[df['type']=='Movie']
fig = px.violin(netflix_movies, x = 'duration', template="plotly_dark", title='Duração em minutos dos Filmes na Netflix',color_discrete_sequence=['red'])
fig.update_layout(yaxis = dict(tickfont = dict(size=15)),xaxis = dict(tickfont = dict(size=15)),title_font=dict(size=20), uniformtext_mode='hide',legend=dict(font=dict(size= 16)))
fig.update_yaxes(title_font=dict(size=16)); fig.update_xaxes(title_font=dict(size=16))

Já em relação aos filmes disponíveis na Netflix a maioria duram em torno de 60 a 145 min

**5.6 As categorias mais assistidas na Netflix ?**

Essa informação ajuda a saber mais sobre o tipo de conteúdo o público prefere assistir levando em conta as categorias. Assim, a Netflix pode decidir quais Gêneros cinematrográficos merecem uma atenção especial, trazendo beneficios a Netflix a longo prazo. Para responder a essa questão vamos plotar um belo gráfico de mapa de árvore! 

In [27]:
df_genre = pd.DataFrame(df.genre.value_counts()).reset_index().rename(columns={'index':'genre', 'genre':'count'})
fig = px.treemap(df_genre, path=[px.Constant("Distribuição de Gêneros"), 'count','genre'])
fig.update_layout(title='Gêneros mais vistos na Netflix', 
                  margin=dict(t=50, b=0, l=70, r=40),
                  plot_bgcolor='#000000', paper_bgcolor='#000000',
                  title_font=dict(size=25, color='#fff'),
                  font=dict(color='#8a8d93'),
                  hoverlabel=dict(bgcolor="#444", font_size=12))

O drama (1600) é o gênero preferido pelo público da Netflix. O segundo gênero preferido é Comédia (1210). E em terceiro vem Documentários. O gêneros com menores audiências são os filmes de ficção ciêntifica (1), esportes (1) e LGBTQ (1).

**5.7 A maioria dos titulos (Filmes e Séries) disponiveis na Netflix foram produzidos em quais países ?**

Para responder a essa questão vamos implementar o deslumbrande gráfico de dispersão geográfica. Sigam-me os bons!

Primeiro vamos gerar o numero titulos disponiveis na netflix em função do país de origem da produção e armazenar num novo dataframe:

In [28]:
# Aqui importamos a biblioteca collections para contabilizar o número de filmes ou series por paises 
import collections as c

# Counter gera o dicionário country_count onde o nome de cada país será uma chave na forma de string 
# e o numero de ocorências de cada string é contabilizada e resultando no seu valor correspondente no dicionario 
country_count = c.Counter(", ".join(df['country']).split(", ")) # Join e Split para contabilizar de forma justa 
                                                                # um conteudo que foi produzido em mais de um país

top_countries = country_count.most_common() # Gera uma lista ordenada a partir do dicionário country_count 
                                            # com base nos valores da chaves, do maior para o menor 
rank = {}
# [::-1] Para que o laço for itere do menor para o maior 
for x in top_countries[::-1]:   
  rank[x[0]] = x[1]             # Ao final do laço é gerado um dicionário ordenado contendo países 
                                # representados pelas chaves e seu respectivos valores que denotam respectivamente
                                # o numero de titulos lá produzidos que podem ser encontrados na netflix

# Gerando um dataframe a partir do dicionário rank e rotulando cada coluna
rank_series = pd.DataFrame(rank.items(),columns=['name', 'count'])  
rank_series

,name,count
0,Montenegro,1
1,East Germany,1
2,Uganda,1
3,Panama,1
4,Sudan,1
...,...,...
123,Canada,445
124,United Kingdom,803
125,Country Unavailable,829
126,India,1046


Agora vamos importar a abreviatura de cada País a partir do popular dataset "countries.csv" para evitar que a vizualização do gráfico seja poluida por tanta informação.

In [29]:
# Importando o dataset já tratado a partir da minha conta no github
url_country = 'https://raw.githubusercontent.com/wanderson42/Portfolio-DS/main/datasets/countries.csv'
df_abrev = pd.read_csv(url_country)

# Mesclando o dataframe anterior (rank_series) com o dataframe de abreviações (df_abrev) 
# Isso só é possível porque a coluna "name" de ambos dataframes são identicas
rank_series = pd.merge(df_abrev, rank_series)
rank_series

,code,name,count
0,AFG,Afghanistan,1
1,ALB,Albania,1
2,DZA,Algeria,3
3,AGO,Angola,1
4,ARG,Argentina,91
...,...,...,...
110,USA,United States,3680
111,URY,Uruguay,14
112,VEN,Venezuela,4
113,VNM,Vietnam,7


Agora finalmente podemos gerar o gráfico de dispersão geográfica para obtermos insights:

In [30]:
fig = px.scatter_geo(rank_series, locations= 'code',
                     hover_name = 'name', size="count", color='count', color_continuous_scale="icefire",range_color=(0, 200),
                     projection="natural earth", size_max=80, template="plotly_dark", title='País de origem das produções disponíveis na Netflix')
fig.update_layout(title_font=dict(size=20), uniformtext_mode='hide')
fig.show()

  Levando em conta um TOP 5: Os Estados Unidos (3683) é a nação onde foram produzidos a maioria dos titulos disponiveis na Netflix. Em segundo, (compondo cerca de 1/3 da produção dos EUA) vem a India (1046), seguidos por Reino Unido (803), Canada (445) e França (393). As estatísticas na America do Sul são bem modestas comparadas ao TOP 5, com menção honrosa para o Brasil (97) e Argentina (91) que são os países sulamericanos que detem a maior produção do subcontinente.

## **Conclusão**

Dado o tamanho e divesidade do dataset da Netflix foi possível explorar, através da fantástica biblioteca Plotly, diversas ferramentas de vizualização de nível básico ao avançado do Plotly, como gráficos de pizza, gráficos de barra, gráfico de funil, gráficos de violino, gráfico de mapa de arvore e gráfico de dispersão geográfica. Aplicamos algumas técnicas de Feature Engineering para aprimorar as vizualizações e com isso foi possível obter varios insights a partir de questões pertinentes.
